In [11]:
import os
import csv
import re

# PGBench Excel {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'tps':''}

# SysBench Excel {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'CPU Total Time':'','Mem Throughput':''}
#see https://wiki.gentoo.org/wiki/Sysbench

# iPerf {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', _________} TODO: Finish the relevant fields

# fio {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'randread/randwrite/seqread/seqwrite runtime':''}

PGBench = [] #For each test run, make a new dict and add it to this list
Sysbench = [] #For each test run, make a new dict and add it to this list
iPerf = [] #For each test run, make a new dict and add it to this list
fio = [] #For each test run, make a new dict and add it to this list

for filename in os.listdir('results'):
    
    
    #PGBench Parser
    if filename[0] == 'p':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('pgbenchResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('pgbenchResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('pgbenchResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('pgbenchResults_.*_.*_.*_(.).*.txt', filename).group(1)
        tps = 0.0
        for line in lines:
            if 'including' in line:
                tps = float(re.search('tps = (.*) \(', line).group(1))
                PGBench.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram, 'tps': tps})#Create dict and add to list
                tps = 0.0#Reset values
                
    CPU = 0.0
    MemOps = 0.0
    MemThroughput = 0.0
    #Sysbench Parser
    if filename[0] == 's':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('sysbenchResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('sysbenchResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('sysbenchResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('sysbenchResults_.*_.*_.*_(.).*.txt', filename).group(1)
        for line in lines:
            if 'events per second' in line:
                CPU = float(re.search('events per second:   (.*)', line).group(1))
            if '4096.00 MiB transferred (' in line:
                MemThroughput = float(re.search('4096.00 MiB transferred \((.*) MiB/sec', line).group(1))
                Sysbench.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram, 'CPU Total Time': CPU, 'Mem Throughput': MemThroughput})
                CPU = 0.0
                MemThroughput = 0.0
                
    #FIO Parser
    if filename[0] == 'f':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('fioResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('fioResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('fioResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('fioResults_.*_.*_.*_(.).*.txt', filename).group(1)
        
        testCount = 0
        testResultDescription = ''
        for line in lines:
            if 'run=' in line:
                testCount=testCount+1
                if (testCount%4)==1:
                    testResultDescription='randread runtime'
                if (testCount%4)==2:
                    testResultDescription='randwrite runtime'
                if (testCount%4)==3:
                    testResultDescription='seqread runtime'
                if (testCount%4)==0:
                    testResultDescription='seqwrite runtime'
                runtime = float( re.search('run=(.*)-', line).group(1))
                fio.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram, testResultDescription: runtime})
                runtime=0.0
                testResultDescription=''

                
def listToCSV(toCSV, filename):
    keys = toCSV[0].keys()
    with open(filename, 'w', newline='')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)        
        
        
        
        
listToCSV(PGBench, 'PGBench.csv')
listToCSV(Sysbench, 'Sysbench.csv')
listToCSV(fio,"FIO.csv")
#listToCSV(iPerf, 'iPerf.csv')


ValueError: dict contains fields not in fieldnames: 'randwrite runtime'